# Tech Layoff Trend Analysis (2020–2025) — EDA + Visualization

This notebook analyzes tech layoffs across companies, industries, and countries. It includes a monthly timeline (Plotly), industry impact, correlation with funding, and a Top 10 companies dashboard.

In [15]:
# Optional: install dependencies if needed (uncomment to run)
# %pip install pandas numpy plotly ipywidgets openpyxl pyarrow
# from google.colab import output; output.enable_custom_widget_manager()  # if in Colab

### Essential Checks for VSCode

In [16]:
import sys, subprocess
print("Kernel:", sys.executable, "\nVersion:", sys.version)
subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "plotly>=5.17", "nbformat>=5.8", "ipywidgets", "kaleido"])

Kernel: c:\Users\anuva\AppData\Local\Programs\Python\Python310\python.exe 
Version: 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]


0

In [17]:
import plotly.io as pio
print(pio.renderers)  # see what's available
#pio.renderers.default = "browser"  # most reliable; opens charts in your web browser
pio.renderers.default = "vscode"  # use only if 'vscode'

Renderers configuration
-----------------------
    Default renderer: 'vscode'
    Available renderers:
        ['plotly_mimetype', 'jupyterlab', 'nteract', 'vscode',
         'notebook', 'notebook_connected', 'kaggle', 'azure', 'colab',
         'cocalc', 'databricks', 'json', 'png', 'jpeg', 'jpg', 'svg',
         'pdf', 'browser', 'firefox', 'chrome', 'chromium', 'iframe',
         'iframe_connected', 'sphinx_gallery', 'sphinx_gallery_png']



In [18]:
%pip install -U pip plotly ipywidgets
import sys; print(sys.executable); print(sys.version)

c:\Users\anuva\AppData\Local\Programs\Python\Python310\python.exe
3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]



### Libraries

In [19]:
import os, re, warnings
from pathlib import Path
from typing import List

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
from IPython.display import display

warnings.filterwarnings('ignore')
pd.options.display.max_columns = 100
pd.options.display.width = 120
pio.templates.default = 'plotly_white'
print('pandas', pd.__version__)
print('numpy', np.__version__)
import plotly; print('plotly', plotly.__version__)


pandas 2.3.2
numpy 2.2.6
plotly 6.3.1


## Locate the dataset
Prefers a file in the repo root whose name contains 'layoff'. Set `DATASET_PATH` to override.

In [20]:
SEARCH_DIRS = [Path('.')]  # prefer root
NAME_HINTS = ['layoff', 'layoffs', 'tech_layoff', 'layoffs_fyi']
EXTS = ['.csv', '.parquet', '.xlsx', '.json']

from typing import List
def find_candidate_paths(max_results: int = 200) -> List[Path]:
    candidates = []
    for root in SEARCH_DIRS:
        for hint in NAME_HINTS:
            for ext in EXTS:
                pattern = f'*{hint}*{ext}' if root == Path('.') else f'**/*{hint}*{ext}'
                try:
                    candidates.extend(root.glob(pattern))
                except Exception:
                    pass
    uniq, seen = [], set()
    for p in candidates:
        try:
            rp = p.resolve()
        except Exception:
            continue
        if not rp.exists() or rp.suffix.lower() not in EXTS:
            continue
        s = str(rp)
        if any(seg in s for seg in ['/.git/', '/node_modules/']):
            continue
        if rp not in seen:
            uniq.append(rp); seen.add(rp)
    def sort_key(p: Path):
        suffix_rank = {'.csv':0,'.parquet':1,'.xlsx':2,'.json':3}.get(p.suffix.lower(),9)
        try:
            size = p.stat().st_size
        except Exception:
            size = 0
        return (suffix_rank, -size, str(p))
    uniq.sort(key=sort_key)
    return uniq[:max_results]

DATASET_PATH = os.environ.get('DATASET_PATH', '').strip()
dataset_candidates: List[Path] = []
if DATASET_PATH:
    dataset_candidates = [Path(DATASET_PATH).resolve()]
if not dataset_candidates:
    dataset_candidates = find_candidate_paths()
print(f'Found {len(dataset_candidates)} candidate file(s).')
for i, p in enumerate(dataset_candidates[:20], 1):
    print(f"{i}. {p}")
if not dataset_candidates:
    print('No dataset found automatically. Set DATASET_PATH, e.g.:')
    print('  $env:DATASET_PATH = \'..\\layoffs.csv\'  (Windows PowerShell)')


Found 1 candidate file(s).
1. C:\Users\anuva\OneDrive\Desktop\Git\Projects\layoffs.csv


In [21]:
if not dataset_candidates:
    raise FileNotFoundError('No layoffs dataset found. Set DATASET_PATH or place layoffs*.csv in repo root.')
dataset_file = dataset_candidates[0]
dataset_file


WindowsPath('C:/Users/anuva/OneDrive/Desktop/Git/Projects/layoffs.csv')

## Load and clean

In [22]:
def coalesce_series(df: pd.DataFrame, cols, default=None) -> pd.Series:
    for c in cols:
        if c in df.columns:
            s = df[c]
            if s.notna().any():
                return s
    return pd.Series(default, index=df.index)

def standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    import re
    df.columns = [re.sub(r'[^a-z0-9]+', '_', str(c).strip().lower()).strip('_') for c in df.columns]
    return df

def load_any(path: Path) -> pd.DataFrame:
    suf = path.suffix.lower()
    if suf == '.csv':
        return pd.read_csv(path)
    if suf == '.parquet':
        return pd.read_parquet(path)
    if suf == '.xlsx':
        try:
            return pd.read_excel(path, engine='openpyxl')
        except Exception:
            return pd.read_excel(path)
    if suf == '.json':
        try:
            return pd.read_json(path, lines=True)
        except ValueError:
            return pd.read_json(path)
    raise ValueError(f'Unsupported file type: {suf}')

raw_df = load_any(dataset_file)
df = standardize_columns(raw_df)

df['company'] = coalesce_series(df, ['company','employer','organization']).astype('string').str.strip()
df['industry'] = coalesce_series(df, ['industry','sector']).astype('string').str.strip()
df['country'] = coalesce_series(df, ['country','location_country','hq_country','location']).astype('string')

def infer_country(val):
    if pd.isna(val):
        return np.nan
    s = str(val)
    parts = [p.strip() for p in s.split(',') if p.strip()]
    if len(parts) >= 1:
        return parts[-1]
    return s
df['country'] = df['country'].map(infer_country)
df['stage'] = coalesce_series(df, ['stage','funding_stage']).astype('string').str.strip()
df['funds_raised_millions'] = pd.to_numeric(coalesce_series(df, ['funds_raised_millions','funds_raised_usd_millions','funds_raised_usd_m','raised','funds_millions']), errors='coerce')
df['total_laid_off'] = pd.to_numeric(coalesce_series(df, ['total_laid_off','laid_off','num_laid_off','n_laid_off','number_laid_off']), errors='coerce')
df['percentage_laid_off'] = pd.to_numeric(coalesce_series(df, ['percentage_laid_off','pct_laid_off','percent_laid_off']), errors='coerce')

date_col = None
for c in ['date','layoff_date','reported_date','announcement_date','month','event_date']:
    if c in df.columns:
        date_col = c
        break
if date_col is None:
    raise ValueError('Could not find a date column among expected names.')

df['date'] = pd.to_datetime(df[date_col], errors='coerce', utc=True).dt.tz_convert(None)
if df['date'].isna().all() and pd.api.types.is_numeric_dtype(df[date_col]):
    df['date'] = pd.to_datetime(df[date_col], unit='s', errors='coerce')

df = df[(df['date'] >= '2020-01-01') & (df['date'] <= '2025-12-31')]
df['year'] = df['date'].dt.year
df['month_period'] = df['date'].dt.to_period('M').dt.to_timestamp()
df['year_month'] = df['date'].dt.to_period('M').astype(str)
df = df[df['total_laid_off'].fillna(0) >= 0]
print('Rows after cleaning:', len(df))
display(df.head(5))


Rows after cleaning: 4174


,company,location,total_laid_off,date,percentage_laid_off,industry,source,stage,funds_raised,country,date_added,funds_raised_millions,year,month_period,year_month
0,Paycom,Oklahoma City,500.0,2025-10-01,NaN,HR,https://www.oklahoman.com/story/business/infor...,Post-IPO,NaN,United States,10/1/2025,NaN,2025,2025-10-01,2025-10
1,Simpl,"Bengaluru,Non-U.S.",80.0,2025-10-01,NaN,Finance,https://inc42.com/buzz/bnpl-startup-simpl-lays...,Series B,$72,India,10/1/2025,NaN,2025,2025-10-01,2025-10
2,Just Eat,"Amsterdam,Non-U.S.",450.0,2025-09-25,NaN,Food,https://www.reuters.com/business/world-at-work...,Acquired,$129,Netherlands,9/29/2025,NaN,2025,2025-09-01,2025-09
3,GamesKraft,"Bengaluru,Non-U.S.",120.0,2025-09-18,NaN,Consumer,https://inc42.com/buzz/now-gameskraft-lays-off...,Unknown,NaN,India,9/25/2025,NaN,2025,2025-09-01,2025-09
4,ZipRecruiter,"Tel Aviv,Non-U.S.",80.0,2025-09-16,NaN,Recruiting,https://www.calcalistech.com/ctechnews/article...,Post-IPO,$769,United States,9/16/2025,NaN,2025,2025-09-01,2025-09


## Aggregations

In [23]:
by_company = (
    df.groupby(['company'], dropna=False, as_index=False)['total_laid_off']
      .sum().sort_values('total_laid_off', ascending=False)
)
by_company_year = df.groupby(['company','year'], as_index=False)['total_laid_off'].sum()
by_country_year = (
    df.groupby(['country','year'], as_index=False)['total_laid_off']
      .sum().sort_values(['year','total_laid_off'], ascending=[True, False])
)
by_industry_year = (
    df.groupby(['industry','year'], as_index=False)['total_laid_off']
      .sum().sort_values(['year','total_laid_off'], ascending=[True, False])
)
monthly = df.groupby('month_period', as_index=False)['total_laid_off'].sum().sort_values('month_period')
display(by_company.head(10))
display(monthly.tail(6))


,company,total_laid_off
1229,Intel,43115.0
1545,Microsoft,30055.0
99,Amazon,27940.0
1537,Meta,24700.0
2090,Salesforce,16525.0
503,Cisco,14521.0
2397,Tesla,14500.0
1036,Google,13547.0
699,Dell,12650.0
2068,SAP,11000.0


,month_period,total_laid_off
62,2025-05-01,10577.0
63,2025-06-01,1606.0
64,2025-07-01,16292.0
65,2025-08-01,6002.0
66,2025-09-01,4152.0
67,2025-10-01,580.0


## Monthly trend

In [24]:
import plotly.io as pio
pio.renderers.default = "vscode"

In [25]:
fig_timeline = px.line(
    monthly, x='month_period', y='total_laid_off', title='Monthly Tech Layoffs (2020–2025)', markers=True
)
fig_timeline.update_layout(yaxis_title='Total Laid Off', xaxis_title='Month', hovermode='x unified')
fig_timeline.show()


## Industry impact

In [26]:
industry_totals = df.groupby('industry', as_index=False)['total_laid_off'].sum().sort_values('total_laid_off', ascending=False)
top_industries = industry_totals.head(20)
fig_industry = px.bar(top_industries, x='industry', y='total_laid_off', title='Industries Most Affected by Layoffs', text='total_laid_off')
fig_industry.update_traces(texttemplate='%{text:.0f}', textposition='outside')
fig_industry.update_layout(xaxis={'categoryorder':'total descending'}, yaxis_title='Total Laid Off')
fig_industry.show()


## Correlation with funding

In [27]:
corr_cols = [c for c in ['total_laid_off','percentage_laid_off','funds_raised_millions'] if c in df.columns]
if corr_cols:
    corr = df[corr_cols].corr(numeric_only=True)
    fig_corr = px.imshow(corr, text_auto=True, aspect='auto', title='Correlation Matrix')
    fig_corr.show()
else:
    print('No numeric columns available for correlation matrix.')

if 'funds_raised_millions' in df.columns:
    scatter_df = df.dropna(subset=['funds_raised_millions','total_laid_off']).copy()
    if len(scatter_df) > 0:
        color_col = 'stage' if 'stage' in df.columns else None
        fig_scatter = px.scatter(scatter_df, x='funds_raised_millions', y='total_laid_off', color=color_col,
                                 title='Layoffs vs. Funds Raised', labels={'funds_raised_millions':'Funds Raised (USD millions)'})
        fig_scatter.update_xaxes(type='log')
        fig_scatter.show()
    else:
        print('Insufficient data for layoffs vs. funds raised scatter.')
else:
    print('Column funds_raised_millions not found; skipping scatter.')


Insufficient data for layoffs vs. funds raised scatter.


## Top 10 Most Affected Companies — interactive

In [28]:
import plotly.express as px
def render_topn_bar(sub_df: pd.DataFrame, topn: int = 10):
    top = (sub_df.groupby('company', as_index=False)['total_laid_off']
             .sum().sort_values('total_laid_off', ascending=False).head(topn))
    if len(top) == 0:
        print('No data for current filters.'); return
    fig = px.bar(top, x='company', y='total_laid_off', title=f'Top {topn} Companies by Layoffs', text='total_laid_off')
    fig.update_traces(texttemplate='%{text:.0f}', textposition='outside')
    fig.update_layout(xaxis={'categoryorder':'total descending'}, yaxis_title='Total Laid Off')
    fig.show()

try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    years = ['All'] + sorted(pd.Series(df['year'].dropna().unique()).astype(int).tolist())
    industries = ['All'] + sorted([str(x) for x in df['industry'].dropna().unique()])
    countries = ['All'] + sorted([str(x) for x in df['country'].dropna().unique()])
    w_year = widgets.Dropdown(options=years, value='All', description='Year:')
    w_industry = widgets.Dropdown(options=industries, value='All', description='Industry:')
    w_country = widgets.Dropdown(options=countries, value='All', description='Country:')
    w_topn = widgets.IntSlider(value=10, min=5, max=30, step=1, description='Top N')
    out = widgets.Output()

    def update_dashboard(*args):
        with out:
            clear_output(wait=True)
            mask = pd.Series(True, index=df.index)
            if w_year.value != 'All': mask &= df['year'] == int(w_year.value)
            if w_industry.value != 'All': mask &= df['industry'] == w_industry.value
            if w_country.value != 'All': mask &= df['country'] == w_country.value
            sub = df[mask] if mask.any() else df.iloc[0:0]
            render_topn_bar(sub, topn=int(w_topn.value))

    for w in [w_year, w_industry, w_country, w_topn]:
        w.observe(update_dashboard, names='value')
    display(widgets.HBox([w_year, w_industry, w_country, w_topn])); display(out); update_dashboard()
except Exception:
    print('ipywidgets not available; showing static Top 10 chart.')
    render_topn_bar(df, topn=10)


Output()

## Summary

In [30]:
from IPython.display import Markdown, display
import pandas as pd
import numpy as np

def _fmt_int(x):
    try:
        return f"{int(x):,}"
    except Exception:
        return "—"

# Overall totals
overall_total = _fmt_int(df['total_laid_off'].sum()) if 'total_laid_off' in df.columns else '—'

# Peak month
peak_month_str, peak_month_total = 'N/A', '—'
if len(monthly) and monthly['total_laid_off'].notna().any():
    peak_idx = monthly['total_laid_off'].idxmax()
    row = monthly.loc[peak_idx]
    try:
        pm = pd.to_datetime(row['month_period'])
        peak_month_str = pm.strftime('%Y-%m')
    except Exception:
        peak_month_str = str(row['month_period'])
    peak_month_total = _fmt_int(row['total_laid_off'])

# Peak year
peak_year_str, peak_year_total = 'N/A', '—'
if 'year' in df.columns and df['year'].notna().any():
    ysum = df.groupby('year')['total_laid_off'].sum().sort_values(ascending=False)
    if len(ysum):
        peak_year_str = str(int(ysum.index[0]))
        peak_year_total = _fmt_int(ysum.iloc[0])

# Top industries
top_industries = []
if 'industry' in df.columns:
    tmp = (df.dropna(subset=['industry'])
             .groupby('industry')['total_laid_off'].sum()
             .sort_values(ascending=False).head(5))
    for name, val in tmp.items():
        top_industries.append(f"- **{name}**: {_fmt_int(val)}")
if not top_industries:
    top_industries = ["- N/A"]

# Top countries
top_countries = []
if 'country' in df.columns:
    tmp = (df.dropna(subset=['country'])
             .groupby('country')['total_laid_off'].sum()
             .sort_values(ascending=False).head(5))
    for name, val in tmp.items():
        top_countries.append(f"- **{name}**: {_fmt_int(val)}")
if not top_countries:
    top_countries = ["- N/A"]

# Top companies
top_companies = []
if 'company' in df.columns:
    tmp = (df.dropna(subset=['company'])
             .groupby('company')['total_laid_off'].sum()
             .sort_values(ascending=False).head(10))
    for i, (name, val) in enumerate(tmp.items(), 1):
        top_companies.append(f"{i}. **{name}** — {_fmt_int(val)}")
if not top_companies:
    top_companies = ["- N/A"]

# Correlation with funds raised
corr_line = "- Correlation unavailable (insufficient data)"
if {'total_laid_off','funds_raised_millions'}.issubset(df.columns):
    cc = df[['total_laid_off','funds_raised_millions']].dropna()
    if len(cc) >= 3:
        r = cc['total_laid_off'].corr(cc['funds_raised_millions'])
        corr_line = f"- Correlation (Pearson) layoffs vs. funds raised: {r:.3f}"

# Totals by stage
stage_lines = []
if 'stage' in df.columns:
    tmp = (df.dropna(subset=['stage'])
             .groupby('stage')['total_laid_off'].sum()
             .sort_values(ascending=False).head(5))
    for name, val in tmp.items():
        stage_lines.append(f"- **{name}**: {_fmt_int(val)}")
if not stage_lines:
    stage_lines = ["- N/A"]

insights_md = f"""## Insights Summary

**Overall**
- **Total layoffs (2020–2025)**: {overall_total}
- **Peak month**: {peak_month_str} — {peak_month_total} layoffs
- **Peak year**: {peak_year_str} — {peak_year_total} layoffs

**Industries most affected (Top 5)**
{chr(10).join(top_industries)}

**Countries most affected (Top 5)**
{chr(10).join(top_countries)}

**Top 10 companies by total layoffs**
{chr(10).join(top_companies)}

**Funding signal**
{corr_line}

**By funding stage (Top 5)**
{chr(10).join(stage_lines)}
"""

display(Markdown(insights_md))

## Insights Summary

**Overall**
- **Total layoffs (2020–2025)**: 770,546
- **Peak month**: 2023-01 — 89,709 layoffs
- **Peak year**: 2023 — 264,220 layoffs

**Industries most affected (Top 5)**
- **Hardware**: 86,528
- **Other**: 82,793
- **Consumer**: 78,596
- **Retail**: 73,721
- **Transportation**: 64,266

**Countries most affected (Top 5)**
- **United States**: 530,135
- **India**: 61,411
- **Germany**: 31,273
- **United Kingdom**: 21,972
- **Netherlands**: 19,875

**Top 10 companies by total layoffs**
1. **Intel** — 43,115
2. **Microsoft** — 30,055
3. **Amazon** — 27,940
4. **Meta** — 24,700
5. **Salesforce** — 16,525
6. **Cisco** — 14,521
7. **Tesla** — 14,500
8. **Google** — 13,547
9. **Dell** — 12,650
10. **SAP** — 11,000

**Funding signal**
- Correlation unavailable (insufficient data)

**By funding stage (Top 5)**
- **Post-IPO**: 451,979
- **Unknown**: 75,160
- **Acquired**: 69,536
- **Series B**: 30,524
- **Series C**: 27,028
